In [2]:
import pandas as pd
df = pd.read_csv("/Users/janbol/Documents/GitHub/RAG/data/test.csv")
df= df[df["Result"].notna()]
data = df.to_dict("records")

In [3]:
len(data)

653

In [14]:
data[:10]

[{'Summary': 'on august 27 2013 employees #1 and #2 of templar inc. a construction company specializing in fiber optic installation and services were working along a highway. the highway speed limit was posted at 55 miles per hour. employee #1 was marking the location of an underground line that ran below the turn lane. employee #2 was next to employee #1 and performing the duties of a flagger. a privately owned vehicle was travelling in the travel/through lane. the vehicle veered to the right entered the turn lane and struck both workers. emergency medical services were called. employee #1 was declared dead at the scene. employee #2 refused emergency medical treatment for the bruises he received when struck.',
  'Result': 1},
 {'Summary': 'at approximately 6:30 a.m. on may 13 2013 employee #1 a foreman regularly employed by integrity rebar placers was operating a rough terrain forklift at a contracted job site in murrieta ca. he was using the forklift to move bundles of steel. employe

In [113]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [114]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [115]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [117]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="incidents",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/c5/p6q0gjzj2ns5lv27n9yp07ph0000gn/T/ipykernel_39385/2185663414.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [122]:
# vectorize!
qdrant.upload_points(
    collection_name="incidents",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Summary"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [162]:
user_prompt = "incidents involving chainsaw and height"

In [163]:
import json
hits = qdrant.search(
    collection_name="incidents",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
) 
for hit in hits:
  print(json.dumps(hit.payload, indent=4), "score:", hit.score)

{
    "Summary": "employee reaches over chainsaw and suffers a wrist laceratio",
    "Result": 1
} score: 0.5600474458120919
{
    "Summary": "on november 14 2013 employee #1 a 33-year-old male journeyman carpenter with torre reich construction was using a skillsaw to cut 1 in. shims from a 2 in. by 4 in. board. as employee #1 went to reposition the saw and start a new cut with the saw blade stopped he reached down with his left hand to pull the bottom guard back so he could place the blade directly against the wood. as employee #1 did this he inadvertently squeezed the trigger with his right hand as he lifted up on the saw. the saw blade started spinning and caused the saw to jump. as the saw jumped the spinning blade struck employee #1's left pinky and amputated the tip.",
    "Result": 1
} score: 0.5467657978803899
{
    "Summary": "at approximately 12:57 p.m. on october 15 2012 employee #1 a foreman/carpenter was installing window trim on residential condominiums. he set a ladder t

In [164]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [166]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are a bot, an construction safety engineer. Your top priority is to help guide fellow engineers to follow safety protocols to reduce incidents at workplace based on the some incident reports"},
        {"role": "user", "content": "create me recommendations to work safely"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
output = completion.choices[0].message.to_dict()
print(output["content"])


**Construction Safety Recommendations**

Based on the incident reports provided, I've identified common safety hazards and recommended measures to prevent similar incidents:

**1. Personal Protective Equipment (PPE)**

* Ensure all employees wear required PPE, including hard hats, safety glasses, ear protection, and steel-toed boots.
* Regularly inspect PPE for damage or wear and tear.

**2. Ladder Safety**

* Ensure ladders are properly inspected and maintained before use.
* Use ladder safety devices, such as ladder levelers or stabilizers, when working at heights.
* Establish a "three-point rule" for ladder use: one hand and two feet, or two hands and one foot, or two feet and one hand.

**3. Chainsaw Safety**

* Always maintain a safe distance from the chainsaw's kickback zone.
* Keep loose clothing and long hair tied back when operating a chainsaw.
* Never reach over or under the chainsaw while it's in operation.

**4. Fall Protection**

* Ensure all employees are properly trained 